## Analisis de datos

#### Creamos un dataframe con los datos de general data y lo fragmentamos y guardamos en un csv por departamento

Obtenemos los datos del csv

In [31]:
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Carga datos en un DataFrame
general_data = pd.read_csv('data.csv', low_memory=False)
general_data.columns = general_data.columns.str.upper()
# Eliminar columnas espurias
general_data =  general_data[~general_data["DEPARTAMENTO"].isin(["QUITO", "MADRID","NEW YORK","NEW JERSEY","FLORIDA"])]
# Convierte la columna FECHA_HECHO a formato datetime
general_data['FECHA_HECHO'] = pd.to_datetime(general_data['FECHA_HECHO'], format="%d/%m/%Y")

Pasamos las conductas delictivas con menos inferencia a una nueva categoria `otros`

In [ ]:
import numpy as np

def replace_by_percentile(data, column, percentile, replacement_value):
    # Calcular el percentil de las frecuencias
    frequencies = data[column].value_counts()
    threshold = np.percentile(frequencies, percentile)
    
    # Crear una copia del DataFrame para evitar modificar el original
    data_copy = data.copy()
    
    # Reemplazar los valores en las filas donde la frecuencia es menor al percentil dado
    data_copy.loc[data_copy[column].map(frequencies) < threshold, column] = replacement_value
    
    return data_copy

modified_data = replace_by_percentile(general_data, "DESCRIPCION_CONDUCTA_CAPTURA", 82, "OTROS")

In [48]:
modified_data[ "DESCRIPCION_CONDUCTA_CAPTURA"].value_counts()

DESCRIPCION_CONDUCTA_CAPTURA
ARTÍCULO 376. TRÁFICO. FABRICACIÓN O PORTE DE ESTUPEFACIENTES                                             203227
ARTÍCULO 239. HURTO PERSONAS                                                                               98813
ARTÍCULO 111. LESIONES PERSONALES                                                                          66558
ARTÍCULO 365. FABRICACIÓN. TRÁFICO. PORTE O TENENCIA DE ARMAS DE FUEGO.ACCESORIOS. PARTES O MUNICIONES     63624
ARTÍCULO 229. VIOLENCIA INTRAFAMILIAR                                                                      46353
                                                                                                           ...  
ARTÍCULO 343. TERRORISMO                                                                                     660
ARTÍCULO 353A. OBSTRUCCION A VIAS PUBLICAS QUE AFECTEN EL ORDEN PUBLICO                                      617
ARTÍCULO 469. ASONADA                                              

Generar un directorio con los datos sin categorizar

In [49]:
def csv_by_filter(data, filter_column, output_folder, out = True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_column, loop_data in data.groupby(filter_column):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_column}.csv")
        if (out):
            print(f"Guardando {output_file}... con {len(loop_data)} filas")
        # Guarda el DataFrame filtrado como CSV
        loop_data.to_csv(output_file, index=False)
    if (out):
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(modified_data, "DEPARTAMENTO", 'departamentos_base', out = False)

Generar un directorio con los datos categorizados

In [52]:
import os
import pandas as pd

def csv_encoded_by_filter(data, filter_column, output_folder, out=True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_value, filtered_data in data.groupby(filter_column):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_value}.csv")
        if out:
            print(f"Guardando {output_file}... con {len(filtered_data)} filas")
        
        # Convierte las columnas de tipo object a tipo category y luego a dummies
        categorical_columns = filtered_data.select_dtypes(include=['object']).columns
        filtered_data = pd.get_dummies(filtered_data, columns=categorical_columns, prefix=categorical_columns, drop_first=False, dtype='int64')

        if(out):
            print("Con el siguiente número de columnas:", len(filtered_data.columns))

        # Guarda el DataFrame filtrado como CSV
        filtered_data.to_csv(output_file, index=False)
    
    if out:
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_encoded_by_filter(modified_data, "DEPARTAMENTO", 'departamentos_codificados', out=True)

Guardando departamentos_codificados\AMAZONAS.csv... con 3042 filas
Con el siguiente número de columnas: 68
Guardando departamentos_codificados\ANTIOQUIA.csv... con 115316 filas
Con el siguiente número de columnas: 325
Guardando departamentos_codificados\ARAUCA.csv... con 4261 filas
Con el siguiente número de columnas: 83
Guardando departamentos_codificados\ATLÁNTICO.csv... con 35313 filas
Con el siguiente número de columnas: 120
Guardando departamentos_codificados\BOLÍVAR.csv... con 34910 filas
Con el siguiente número de columnas: 167
Guardando departamentos_codificados\BOYACÁ.csv... con 27861 filas
Con el siguiente número de columnas: 316
Guardando departamentos_codificados\CALDAS.csv... con 29073 filas
Con el siguiente número de columnas: 126
Guardando departamentos_codificados\CAQUETÁ.csv... con 10663 filas
Con el siguiente número de columnas: 103
Guardando departamentos_codificados\CASANARE.csv... con 9640 filas
Con el siguiente número de columnas: 111
Guardando departamentos_codif